In [167]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
def cv_show(name,img):
    cv2.startWindowThread()
    cv2.imshow(name,img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)
template=cv2.imread('bank3.png')
cv_show('template',template)

In [51]:
def sort_my_contours(contours):
    i=0
    reverse=False
    bondingBoxes=[cv2.boundingRect(c) for c in contours]
    (contours,bondingBoxes)=zip(*sorted(zip(contours,bondingBoxes),
    key=lambda b:b[1][i],reverse=reverse))
    return contours

In [175]:
template=cv2.imread('bank1.jpeg')
ref=cv2.cvtColor(template,cv2.COLOR_BGR2GRAY)
ref=cv2.threshold(ref,127,255,cv2.THRESH_BINARY)[1]
contours,hierachy=cv2.findContours(ref,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
draw_gray=template.copy()
# res3=cv2.drawContours(draw_gray,contours,-1,(0,0,255),3)
sorted_contours=sort_my_contours(contours)
res3=cv2.drawContours(draw_gray,sorted_contours,0,(0,0,255),3)
cv_show('fourth',res3)

In [148]:
digits={}
for(i,c) in enumerate(sorted_contours):
    (x,y,w,h)=cv2.boundingRect(c)
    roi=ref[y:y+h,x:x+w]
    roi=cv2.resize(roi,(57,88))
    digits[i]=roi 

In [198]:
rectKernel=cv2.getStructuringElement(cv2.MORPH_RECT,(9,5))
sqKernel=cv2.getStructuringElement(cv2.MORPH_RECT,(5,5))
img_bank=cv2.imread('bank2.png')
img_bank=cv2.resize(img_bank,(506,321))
gray=cv2.cvtColor(img_bank,cv2.COLOR_BGR2GRAY)
tophat=cv2.morphologyEx(gray,cv2.MORPH_TOPHAT,rectKernel)
cv_show('tophat',tophat)
sobelx=cv2.Sobel(gray,cv2.MORPH_TOPHAT,1,0,ksize=-1)
sobelx=np.absolute(sobelx)
(maxVal,minVal)=(np.max(sobelx),np.min(sobelx))
sobelx=(255*((sobelx-minVal)/(maxVal-minVal)))
sobelx=sobelx.astype('uint8')
closing=cv2.morphologyEx(sobelx,cv2.MORPH_CLOSE,rectKernel)
thresh=cv2.threshold(sobelx,0,255,cv2.THRESH_BINARY|cv2.THRESH_OTSU)[1]
closing=cv2.morphologyEx(thresh,cv2.MORPH_CLOSE,rectKernel)
close=cv2.morphologyEx(closing,cv2.MORPH_CLOSE,rectKernel)
contours,hierachy=cv2.findContours(close,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
draw_gray=img_bank.copy()
res=cv2.drawContours(draw_gray,contours,-1,(0,0,255),3)
loc=[]
for (i,c) in enumerate(contours):    
    (x,y,w,h)=cv2.boundingRect(c)
    if(h>22 and h<30 and y>100):
        loc.append((x,y,w,h))
loc=sorted(loc,key=lambda x:x[0])
output=[]
for(i,(gx,gy,gw,gh)) in enumerate(loc):
    groupOutput=[]
    group = gray[gy-5:gy+gh+5,gx-5:gx+gw+5]
    group=cv2.threshold(group,210,255,cv2.THRESH_BINARY)[1]
    group_contours,hierachy=cv2.findContours(group,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    digit_groups=sort_my_contours(group_contours)
    count=0
    for c in digit_groups:
        count+=1
        (x,y,w,h)=cv2.boundingRect(c)
        roi=group[y:y+h,x:x+w]
        roi=cv2.resize(roi,(57,88))
        scores=[]
        for(digit,digitROI) in digits.items():
            result=cv2.matchTemplate(roi,digitROI,cv2.TM_CCOEFF)
            (_,score,_,_)=cv2.minMaxLoc(result)
            scores.append(score)
        groupOutput.append(str(np.argmax(scores)))   
        cv2.rectangle(img_bank,(gx-5,gy-5),(gx+gw+5,gy+gh+5),(0,0,255),1)
        cv2.putText(img_bank,"".join(groupOutput),(gx,gy-15),cv2.FONT_HERSHEY_SIMPLEX,0.65,(0,0,255),2)
    output.extend(groupOutput)
cv_show('img_bank',img_bank)
print("Card #{}".format("".join(output)))

Card #62293988888888888


In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
def cv_show(name,img):
    cv2.startWindowThread()
    cv2.imshow(name,img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)
def sort_my_contours(contours):
    i=0
    reverse=False
    bondingBoxes=[cv2.boundingRect(c) for c in contours]
    (contours,bondingBoxes)=zip(*sorted(zip(contours,bondingBoxes),
    key=lambda b:b[1][i],reverse=reverse))
    return contours
template=cv2.imread('bank4.jpg')
ref=cv2.cvtColor(template,cv2.COLOR_BGR2GRAY)
ref=cv2.threshold(ref,20,255,cv2.THRESH_BINARY_INV)[1]
cv_show('ref',ref)
contours,hierachy=cv2.findContours(ref,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
draw_gray=template.copy()
# res3=cv2.drawContours(draw_gray,contours,-1,(0,0,255),3)
sorted_contours=sort_my_contours(contours)
res3=cv2.drawContours(draw_gray,sorted_contours,0,(0,0,255),3)
digits={}
for(i,c) in enumerate(sorted_contours):
    (x,y,w,h)=cv2.boundingRect(c)
    roi=ref[y:y+h,x:x+w]
    roi=cv2.resize(roi,(57,88))
    digits[i]=roi 
    cv_show('roi',roi)
rectKernel=cv2.getStructuringElement(cv2.MORPH_RECT,(9,5))
sqKernel=cv2.getStructuringElement(cv2.MORPH_RECT,(5,5))
img_bank=cv2.imread('bank1.jpg')
img_bank=cv2.resize(img_bank,(506,321))
gray=cv2.cvtColor(img_bank,cv2.COLOR_BGR2GRAY)
tophat=cv2.morphologyEx(gray,cv2.MORPH_TOPHAT,rectKernel)
cv_show('tophat',tophat)
sobelx=cv2.Sobel(gray,cv2.MORPH_TOPHAT,1,0,ksize=-1)
sobelx=np.absolute(sobelx)
(maxVal,minVal)=(np.max(sobelx),np.min(sobelx))
sobelx=(255*((sobelx-minVal)/(maxVal-minVal)))
sobelx=sobelx.astype('uint8')
closing=cv2.morphologyEx(sobelx,cv2.MORPH_CLOSE,rectKernel)
thresh=cv2.threshold(sobelx,0,255,cv2.THRESH_BINARY|cv2.THRESH_OTSU)[1]
closing=cv2.morphologyEx(thresh,cv2.MORPH_CLOSE,rectKernel)
close=cv2.morphologyEx(closing,cv2.MORPH_CLOSE,rectKernel)
contours,hierachy=cv2.findContours(close,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
draw_gray=img_bank.copy()
res=cv2.drawContours(draw_gray,contours,-1,(0,0,255),3)
cv_show('res',res)
loc=[]
for (i,c) in enumerate(contours):    
    (x,y,w,h)=cv2.boundingRect(c)
    if(h>22 and h<40 and y>100):
        loc.append((x,y,w,h))
loc=sorted(loc,key=lambda x:x[0])
output=[]
for(i,(gx,gy,gw,gh)) in enumerate(loc):
    groupOutput=[]
    group = gray[gy-5:gy+gh+5,gx-5:gx+gw+5]
    group=cv2.threshold(group,210,255,cv2.THRESH_BINARY)[1]
    group_contours,hierachy=cv2.findContours(group,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    if(group_contours!=()):
        digit_groups=sort_my_contours(group_contours)
        count=0
        for c in digit_groups:
            count+=1
            (x,y,w,h)=cv2.boundingRect(c)
            roi=group[y:y+h,x:x+w]
            roi=cv2.resize(roi,(57,88))
            scores=[]
            for(digit,digitROI) in digits.items():
                result=cv2.matchTemplate(roi,digitROI,cv2.TM_CCOEFF)
                (_,score,_,_)=cv2.minMaxLoc(result)
                scores.append(score)
            groupOutput.append(str(np.argmax(scores)))   
            cv2.rectangle(img_bank,(gx-5,gy-5),(gx+gw+5,gy+gh+5),(0,255,0),1)
            cv2.putText(img_bank,"".join(groupOutput),(gx,gy-15),cv2.FONT_HERSHEY_SIMPLEX,0.65,(255,0,0),2)
        output.extend(groupOutput)
cv_show('img_bank',img_bank)
print("Card #{}".format("".join(output)))

Card #4000123456789010
